# Métodos de Representación y Visualización Espacial HHDD

Trabajo de evaluación curso 2021-22

Profesor Manuél Díaz Ordoñez


## Descripción

El presente [Jupyter notebook](https://jupyter.org/) realiza el procesado de datos parte del trabajo de evaluación de la asignatura. El ejercicio consiste en acceder a [Soundtoll Database](http://www.soundtoll.nl/index.php/en/over-het-project/str-online), seleccionar una mercancía y periodo de tiempo y descargar en formato CSV la información relativa a _cargoes_ y _passages_.

### Parámetros de entrada
Los parámetros de búsqueda en la Soundtoll Database son:
* mercancía: **ámbar**, la cadena de búsqueda usada ha sido `B*rnste*n`, la cual cubre _Barnsteen_ (danés), _Bernstein_ (neerlandés) y diversas variaciones.
* periodo: **1634-1857** 

Los archivo CSV resultantes están almacenados en el directorio `data/`.

### Funcionalidades
El código realiza las siguientes operaciones sobre los archivos CSV:
* genera un campo _fecha_ combinando los campos _día_, _month_ and _year_ del dataset _passages_
* filtra los registros del dataset _cargoes_ asociados con ámbar en cualquiera de sus traducciones soportadas (DK, NL)
* elimina los registros asociados a puertos excluidos del estudio
* hace uniformes los nombres de puertos basado en un conjunto de reglas preestablecidas, este paso elimina la variabilidad presente en los puertos
* selecciona unidades de peso soportadas (e.g. skippund, pund, ..) y las convierte a Kg
* asocia las coordenadas de longitud y latitud a cada uno de los puertos
* combina los datos de los dataset _cargoes_ y _passages_
* genera dos nuevos datasets basados en el puerto de salida y el de llegada de la mercancía respectivamente, cada dataset contiene el sumatorio de kilos de mercancía para cada uno de los puertos

## Código fuente


### Librerías y funciones axuliares

In [74]:
import json
import re
import pandas as pd

from datetime import datetime
from fractions import Fraction
from geopy import geocoders  


def clean_port(port, dct):
    """
    Check a given port agaisnt the dictionary with name rules.
    Returns a corrected name if any rule applies
    
    :param port: port name, e.g. Ambsterdam
    :param dct: rule dictionary with naming regular expressions
    :returns: corrected port name based on rule dictionary
    """
    for key, regex in dct.items():
        if bool(re.search(regex, port)):
            return key
    return port


def filter_amber(x):
    """
    Check if string x matches the word "amber" in any of the
    supported languagles (Danish, Dutch and English)
    
    :param: x string
    :returns: True if it maches "amber" in Danish, Dutch or English
    """
    # rule for English
    re_amber = r'ambe'
    # rule for Danish and Dutch
    re_bersteen = r'b.rnste.*n'
    return bool(re.search(re_amber, x.lower())) or bool(re.search(re_bersteen, x.lower()))


def geolocate_ports(ports):
    """
    Calculate geographical coordinates for a list of cities (strings).
    
    :param ports: list of strings
    :returns: dictionary where each city (key) has assciated its coordinates
              as a tuple. If a city is not found, coordinates are (None, None)
              
    :example:
        ports = ['Amsterdam', 'Rotterdam']
        coor = geolocate_ports(ports)
        print(coor)
        > {
        >   'Amsterdam': (x1, y1),
        >   'Rotterdam': (x2, y2)
        > }
    """
    gn = geocoders.Nominatim(user_agent='fakeusername')
    coordinates  = {}
    
    for port in ports:
        coor = gn.geocode(port)
        if coor is not None:
            coordinates[port] = (coor.latitude, coor.longitude)
        else:
            coordinates[port] = (None, None)
        
    return coordinates


def fraction_to_float(text):
    """
    Checks if a variable is a string containing a fraction expression
    (e.g. "1 1/2"). If so it calculates the equivalent float value.
    
    :param text: value to check
    :returns: float value
    """
    if isinstance(text, str):
        total = [Fraction(part) for part in text.split(' ')]
        return float(sum(total))
    else:
        return text
    

### Archivos CSV y campos asociados

In [75]:
# CSV files to be processed
file_cargo = 'data/cargoes_Advanced_search_results_values_B%rnste%n__1634_1857_.csv'
file_passage = 'data/passages_Advanced_search_results_values_B%rnste%n__1634_1857_.csv'

# translation NL-EN for cargo headers
fields_cargo = {
    'id_doorvaart': 'id',
    'van': 'from',
    'naar': 'to',
    'maat': 'unit', 
    'aantal': 'amount',
    'soort': 'type'
}
# list of NL cargo headers only
fields_cargo_nl = fields_cargo.keys()

# translation NL-EN for passage headers
fields_passage = {
    'id_doorvaart': 'id',
    'dag': 'day',
    'maand': 'month',
    'jaar': 'year'
}
# list of NL passage headers only
fields_passage_nl = fields_passage

### Nombres de puertos
* puertos a ignorar: no se corresponden con localizaciones concretas, otros motivos
* reglas para procesar y uniformar nombres de puertos (expresiones regulares)

In [76]:
# port names to be removed
ports_remove = ['-', 'Østersøen', 'Franckeriige', 'Habel de Graas', 'Westerwig', 'Der Liebau', 'Liebau', 'Norge']

ports_regex = {
    'Amsterdam': r'Amb?s.*',
    'Konigsberg': r'Cønn?i.+|K.nn?i?s?.+',
    'Danzig': r'Dan.+',
    'Dunkirk': r'D.nk.rchen',
    'Emden': r'Emb?den',
    'Gottenborg': r'Got.enborg',
    'Habel de Graas': r'Habel de Gra.s',
    'Helsingør': r'Helsingø.?r',
    'Hull': r'Hul?',
    'Copenhagen': r'K.øbenha..',
    'Landskrona': r'Landscrona',
    'Leeuwarden': r'L.ur?w.+en',
    'London': r'L[o|u]nn?d',
    'Montrose': r'Montrosse?',
    'Newcastle': r'Nycast.*',
    'Petersburg': r'Petersb.rg',
    'Rugenwalde': r'R.genwalde',
    'Stockholm': r'Stock.*',
    'Vedbæk': r'Wedbek',
    'Dundee': r'Dundie',
    'Trondheim': r'Tronhiem',
    'Ramsgate': r'Romansgate',
    'Klaipėda': r'Der Memel',
}

use_coordinates_cache = True

### Tabla de conversión de unidades locales de medida a Kilogramos

In [77]:
unit_to_kg = {
    # https://www.sizes.com/units/pund.htm
    'Pund': 0.5,
    # https://www.sizes.com/units/skippund.htm
    'Skippund': 160.076,
    # https://sv.wikipedia.org/wiki/Lispund
    # https://www.sizes.com/units/lispund.htm
    'Lispund': 8.003,
    # https://www.sizes.com/units/centner.htm
    'Centner': 100,
}

### Cargar datos desde archivos CSV

In [78]:
# cargoes
cargoes_raw = pd.read_csv(file_cargo, header=0, usecols=fields_cargo_nl, sep=';')
cargoes_raw = cargoes_raw.rename(columns=fields_cargo)

# passages
passages_raw = pd.read_csv(file_passage, header=0, usecols=fields_passage_nl, sep=';')
passages_raw = passages_raw.rename(columns=fields_passage)

### Construir campo fecha

Unimos los campos `year`, `month` y `day` en un único campo `date` (Fecha)

In [79]:
# create new column date combining several columns
passages_raw['date'] = passages_raw[['year', 'month', 'day']].apply(
    lambda x: datetime(x[0], x[1], x[2]),
    axis=1
)
# remove columns day, montgh and year
passages = passages_raw.drop(columns=['day', 'month', 'year'])

### Seleccionar carga de tipo "ámbar"

Filtramos el dataset `cargoes_raw` seleccionando únicamente las filas del tipo _ámbar_. La función `filter_amber` permite determinar si una cadena de texto se corresponde con la mercancía _ámbar_ en alguno de los idiomas soportados (danés, inglés y neerlándés).

In [80]:
# logic vector, indicates if fow is "amber" type or not
filter_vector = cargoes_raw['type'].apply(filter_amber)
# keep only rows of "amber" type
cargoes_raw = cargoes_raw[filter_vector]

### Procesar nombres de puertos

In [81]:
cargoes_raw['port_from'] = cargoes_raw[['from']].apply(lambda x: clean_port(x[0], ports_regex), axis=1)
cargoes_raw['port_to'] = cargoes_raw[['to']].apply(lambda x: clean_port(x[0], ports_regex), axis=1)
# remove original columns, not needed any more
cargoes = cargoes_raw.drop(['from', 'to'], axis=1)

### Filtrar unidades de medida soportadas

In [82]:
cargoes = cargoes[cargoes['unit'].isin(unit_to_kg.keys())]
cargoes = cargoes[~cargoes['port_from'].isin(ports_remove)]
cargoes = cargoes[~cargoes['port_to'].isin(ports_remove)]

### Procesar fracciones
Algunas cantidades están expresadas como fracciones (i.e. '1 3/4'). Convertimos esta representación a formato decimal.

In [83]:
# from fractions to float
cargoes['amount'] = cargoes['amount'].apply(fraction_to_float)
cargoes['kg'] = cargoes[['amount', 'unit']].apply(lambda x: x[0] * unit_to_kg[x[1]], axis=1)

### Agrupar datos por viaje
Algunos viajes presentan varias entradas para la mercancía _ámbar_ en distintas unidades de medida. Una vez que todas las unidades de medida han sido convertidas a kilogramos, es posible sumar estas cantidades, obteniendo el total de kilogramos transportados en un determinado viaje

In [84]:
# group by id, port_from and port_to, calculate sum of kg
# original units and amounts will be lost during the aggregation
cargoes_summary = cargoes.groupby(['id', 'port_from', 'port_to'])['kg'].sum()
# it is required to reset indexes
cargoes_summary = cargoes_summary.reset_index()

### Calcular / cargar coordenadas

Cálculo de coordenadas puede ser lento. Proporcionamos un dicionario precalculado.

Re-calcular coordendadas sólo requiere asigar valor `False` a la variable `use_coordinates_cache`.

In [85]:
# list of all the processed port names in cargoes dataset
ports_filtered = set.union(set(cargoes['port_from'].unique()), set(cargoes['port_to'].unique()))

use_coordinates_cache = True
if use_coordinates_cache:
    # load coordinates cache
    with open('./coordinates.json', 'r') as file_json:
        coordinates = json.load(file_json)
else:
    coordinates = geolocate_ports(list(ports_filtered))
    with open('./coordinates.json', 'w') as file_json:
        json.dump(coordinates, file_json)
    
cargoes_summary['lat_from'] = cargoes_summary['port_from'].apply(lambda x: coordinates[x][0])
cargoes_summary['lon_from'] = cargoes_summary['port_from'].apply(lambda x: coordinates[x][1])

cargoes_summary['lat_to'] = cargoes_summary['port_to'].apply(lambda x: coordinates[x][0])
cargoes_summary['lon_to'] = cargoes_summary['port_to'].apply(lambda x: coordinates[x][1])

### Generar dataset final

In [86]:
table = pd.merge(cargoes_summary, passages, on='id')
table.to_csv('amber.csv')

In [88]:
table

,id,port_from,port_to,kg,lat_from,lon_from,lat_to,lon_to,date
0,134935,Malmøe,Gottenborg,2.0,55.605293,13.000157,60.902828,11.806039,1782-05-29 00:00:00
1,222683,Stettin,Amsterdam,24.0,53.429681,14.592913,52.372760,4.893604,1776-12-13 00:00:00
2,231440,Malmøe,Gottenborg,2.5,55.605293,13.000157,60.902828,11.806039,1779-07-14 00:00:00
3,238566,Rugenwalde,Amsterdam,150.0,54.429150,16.403846,52.372760,4.893604,1775-07-17 00:00:00
4,242760,Danzig,Amsterdam,22.0,54.361193,18.628609,52.372760,4.893604,1769-09-20 00:00:00
...,...,...,...,...,...,...,...,...,...
968,1752582,Danzig,Amsterdam,20.0,54.361193,18.628609,52.372760,4.893604,1700-11-08 00:00:00
969,5087883,Danzig,Amsterdam,16.5,54.361193,18.628609,52.372760,4.893604,1733-09-23 00:00:00
970,10000021,Danzig,London,159.0,54.361193,18.628609,51.507322,-0.127647,1751-05-23 00:00:00
971,10000315,Danzig,Amsterdam,90.0,54.361193,18.628609,52.372760,4.893604,1757-10-12 00:00:00


In [90]:
port_from = table.groupby(['port_from', 'lat_from', 'lon_from'])['kg'].sum()
port_from = port_from.reset_index()
port_from.to_csv('amber_port_origin.csv')

port_to = table.groupby(['port_to', 'lat_to', 'lon_to'])['kg'].sum()
port_to = port_to.reset_index()
port_to.to_csv('amber_port_destination.csv')

In [91]:
port_from

,port_from,lat_from,lon_from,kg
0,Amsterdam,52.372760,4.893604,2321.102000
1,Colberg,54.176914,15.575964,112.500000
2,Danzig,54.361193,18.628609,424873.212800
3,Gottenborg,60.902828,11.806039,5.000000
4,Hamborg,53.550341,10.000654,495.003067
5,Hull,53.743572,-0.339476,4976.500000
6,Klaipėda,55.712753,21.135047,135.000000
7,Konigsberg,54.710128,20.510584,3367.304000
8,Livorno,42.790219,10.340281,1872.889200
9,London,51.507322,-0.127647,188.415667


In [92]:
port_to

,port_to,lat_to,lon_to,kg
0,Amsterdam,52.372760,4.893604,380944.195800
1,Antwerpen,51.221110,4.399708,1097.625000
2,Bordeaux,44.841225,-0.580036,120.000000
3,Bremen,53.075820,8.807165,779.500000
4,Copenhagen,55.686724,12.570072,853.738667
5,Danzig,54.361193,18.628609,530.991667
6,Diep.,49.924618,1.079144,15.000000
7,Dundee,56.460594,-2.970190,50.000000
8,Emden,53.367054,7.205830,187.500000
9,Engelland,52.525621,6.250738,10.000000
